# **TEST**

In [2]:
import json
sys.path.append('../')
from utilities import utility_matrix
import pandas as pd
import itertools
from tqdm import tqdm
import numpy as np
import scipy.spatial.distance as distance

In [3]:
# open dataset
with open('../../data/full_data2.json', 'r') as file:
    data = json.load(file)

In [94]:
conditions_df = pd.DataFrame(data['Conditions'])[:20]
therapies_df = pd.DataFrame(data['Therapies'])[:10]
patients_df = pd.DataFrame(data['Patients'])

In [5]:
pat_0_mat = utility_matrix("pat_0")
pat_0_mat

,ther_0,ther_1,ther_2,ther_3,ther_4,ther_5,ther_6,ther_7,ther_8,ther_9
cond_0,0,0,0,0,0,0,0,0,0,0
cond_1,0,0,0,59,7,0,0,37,4,0
cond_2,0,0,0,0,0,75,0,96,0,89
cond_3,0,0,82,0,0,0,0,45,2,0
cond_4,0,0,0,0,62,68,63,0,0,0
cond_5,0,0,0,98,0,0,0,0,0,0
cond_6,0,0,0,0,0,0,0,0,0,0
cond_7,0,0,0,0,0,0,0,0,0,0
cond_8,0,0,0,0,0,0,0,0,0,0
cond_9,0,0,0,0,0,0,0,0,0,0


In [6]:
pat_cured = []
for i in patients_df.id.values:
    if any(d['kind'] == 'cond_2' and d['cured'] != None for d in patients_df[patients_df.id == i].conditions.values[0]):
        #print(i)
        
        pat_cured.append(i)

In [7]:
pat_cured

['pat_0', 'pat_4', 'pat_6']

In [8]:
u_mat = utility_matrix("pat_1")
vect_1 = u_mat.to_numpy()

In [9]:
utility_matrix("pat_0").to_numpy()

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 59,  7,  0,  0, 37,  4,  0],
       [ 0,  0,  0,  0,  0, 75,  0, 96,  0, 89],
       [ 0,  0, 82,  0,  0,  0,  0, 45,  2,  0],
       [ 0,  0,  0,  0, 62, 68, 63,  0,  0,  0],
       [ 0,  0,  0, 98,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [89,  0,  0,  0,  0,  0,  0,  0,  0, 33],
       [ 0,  0,  0,  0,  0,  0, 96,  0,  0,  0],
       [81,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 37,  0, 82,  0, 26,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [67,  0,  0,  0,  0,  0, 91,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 14,  0, 30,  0, 88, 24]], dtype=int64)

In [10]:
cosine_sim = []

for i in tqdm(pat_cured, desc="Caclulating cosine distances"):
    pat = utility_matrix(i)
    mat = pat.to_numpy()
    cosine_sim_pat_0 = 1 - distance.cosine(vect_1.flatten(), mat.flatten())
    cosine_sim.append(cosine_sim_pat_0)
    

Caclulating cosine distances: 100%|██████████| 3/3 [00:00<00:00, 30.95it/s]


In [11]:
dff = pd.DataFrame({"pat_cured" : pat_cured, 
                    "cosine_similarity": cosine_sim})
dff.sort_values("cosine_similarity", ascending=False)

,pat_cured,cosine_similarity
1,pat_4,0.156972
0,pat_0,0.108105
2,pat_6,0.066371


In [12]:
reccomended_ther = pd.DataFrame(columns = ['id', 'name', 'type'])
succ_cond_id = list(filter(lambda d: d['kind'] == "cond_2", patients_df[patients_df.id == "pat_0"].conditions.values[0]))[0]['id'] 
ther_id = list(filter(lambda d: d['condition'] == succ_cond_id and d['successful']>90, patients_df[patients_df.id == "pat_0"].trials.values[0]))[0]['therapy']
reccomended_ther = reccomended_ther.append(therapies_df[therapies_df.id == ther_id], ignore_index = True)
reccomended_ther

C:\Users\elypa\AppData\Local\Temp\ipykernel_18264\1383727915.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reccomended_ther = reccomended_ther.append(therapies_df[therapies_df.id == ther_id], ignore_index = True)


,id,name,type
0,ther_7,Arthroscopy,Surgical procedures


---
## Complete algorithm

In [36]:

def my_program(data, patient_id, condition):
    reccomended_ther = pd.DataFrame(columns = ['id', 'name', 'type', 'details'])

    # open dataset
    # with open(dataset_path, 'r') as file:
    #     data = json.load(file)
    patients_df = pd.DataFrame(data['Patients'])
    conditions_df = pd.DataFrame(data['Conditions'])
    therapies_df = pd.DataFrame(data['Therapies'])
    # check if patients and condition exists
    if patient_id not in patients_df.id.values:
        print("Patient ID is not in the data")
    else:
        if condition not in conditions_df.name.values:
            print("Condition is not in the data")
        else:
            # check if condition was already succesfully cured by a therapy (for the given patient)
            cond_id = conditions_df[conditions_df.name==condition].id.values[0]
            if any(d['kind'] == cond_id and d['cured'] != None for d in patients_df[patients_df.id == patient_id].conditions.values[0]):
                
                succ_cond_id = list(filter(lambda d: d['kind'] == cond_id, patients_df[patients_df.id == patient_id].conditions.values[0]))[0]['id'] 
                ther_id = list(filter(lambda d: d['condition'] == succ_cond_id and d['successful']>90, patients_df[patients_df.id == patient_id].trials.values[0]))[0]['therapy']

                ther_row = therapies_df[therapies_df.id == ther_id]
                ther_row['details'] = f"cured with success on itself" 
                reccomended_ther = reccomended_ther.append(ther_row, ignore_index = True)
                #return reccomended_ther
            # else:
            # find cured patients subset
            pat_cured = []
            for i in patients_df.id.values:
                if any(d['kind'] == cond_id and d['cured'] != None for d in patients_df[patients_df.id == i].conditions.values[0]):
                    #print(i)
                    
                    pat_cured.append(i)
            # build patient utility matrix
            patient_id_u_mat = utility_matrix(patient_id).to_numpy()
            # build cured patient utility matrix and calculate cosine similarity with patient id matrix
            cosine_sim = []

            for i in tqdm(pat_cured, desc="Caclulating cosine distances"):
                cured_u_mat = utility_matrix(i).to_numpy()
                cosine_sim_pat = 1 - distance.cosine(patient_id_u_mat.flatten(), cured_u_mat.flatten())
                cosine_sim.append(cosine_sim_pat)
            
            cos_sim_df = pd.DataFrame({"pat_cured" : pat_cured, 
                "cosine_similarity": cosine_sim})
            cos_sim_df_sort = cos_sim_df.sort_values("cosine_similarity", ascending=False)
            
            # extract therapies that cured the most similar patients
            for pat in cos_sim_df_sort.pat_cured.values:
                if len(reccomended_ther) < 5:
                    succ_cond_id = list(filter(lambda d: d['kind'] == cond_id, patients_df[patients_df.id == pat].conditions.values[0]))[0]['id'] 
                    ther_id = list(filter(lambda d: d['condition'] == succ_cond_id and d['successful']>90, patients_df[patients_df.id == pat].trials.values[0]))[0]['therapy']
                    
                    similarity = cos_sim_df_sort[cos_sim_df_sort.pat_cured == pat]['cosine_similarity'].values[0]
                    ther_row = therapies_df[therapies_df.id == ther_id]
                    ther_row['details'] = f"cured with success on {pat} ({similarity} cosine similarity with {patient_id})" 
                    # add only therapies that are not in the reccomandation yet
                    if ther_id not in reccomended_ther.id.values:
                        reccomended_ther = reccomended_ther.append(ther_row, ignore_index = True)

    return reccomended_ther
                

ModuleNotFoundError: No module named 'md5'

In [24]:
i = my_program(data, "pat_6", "Acute cholecystitis")

C:\Users\elypa\AppData\Local\Temp\ipykernel_18264\3680822559.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reccomended_ther = reccomended_ther.append(therapies_df[therapies_df.id == ther_id], ignore_index = True)
Caclulating cosine distances: 100%|██████████| 3/3 [00:00<00:00, 36.97it/s]
C:\Users\elypa\AppData\Local\Temp\ipykernel_18264\3680822559.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reccomended_ther = reccomended_ther.append(therapies_df[therapies_df.id == ther_id], ignore_index = True)


In [25]:
i

,id,name,type
0,ther_0,Abortion,Surgical procedures
1,ther_7,Arthroscopy,Surgical procedures
2,ther_4,Antibiotics,Medicines and medical aids


In [18]:
reccomended_ther = pd.DataFrame(columns = ['id', 'name', 'type'])
for pat in dff.pat_cured.values[:5]:   
    succ_cond_id = list(filter(lambda d: d['kind'] == "cond_2", patients_df[patients_df.id == pat].conditions.values[0]))[0]['id'] 
    ther_id = list(filter(lambda d: d['condition'] == succ_cond_id and d['successful']>90, patients_df[patients_df.id == pat].trials.values[0]))[0]['therapy']
    if ther_id not in reccomended_ther.id.values:
        reccomended_ther = reccomended_ther.append(therapies_df[therapies_df.id == ther_id], ignore_index = True)
reccomended_ther

C:\Users\elypa\AppData\Local\Temp\ipykernel_18264\3435266761.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reccomended_ther = reccomended_ther.append(therapies_df[therapies_df.id == ther_id], ignore_index = True)


,id,name,type
0,ther_7,Arthroscopy,Surgical procedures
1,ther_4,Antibiotics,Medicines and medical aids
2,ther_0,Abortion,Surgical procedures


## Try with huge data

In [27]:
with open('../../data/full_data.json', 'r') as file:
    data = json.load(file)

In [34]:
i = my_program(data, "pat_6", "Acute cholecystitis")

Caclulating cosine distances: 100%|██████████| 135/135 [03:15<00:00,  1.44s/it]
C:\Users\elypa\AppData\Local\Temp\ipykernel_3484\2100057466.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ther_row['details'] = f"cured with success on {pat} ({similarity} cosine similarity with {patient_id})"
C:\Users\elypa\AppData\Local\Temp\ipykernel_3484\2100057466.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reccomended_ther = reccomended_ther.append(ther_row, ignore_index = True)
C:\Users\elypa\AppData\Local\Temp\ipykernel_3484\2100057466.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [35]:
i

,id,name,type,details
0,ther_60,Statins,Medicines and medical aids,cured with success on pat_5698 (0.052977777389...
1,ther_6,"Antiplatelets, clopidogrel",Medicines and medical aids,cured with success on pat_5460 (0.041568813352...
2,ther_10,Beta-blockers,Medicines and medical aids,cured with success on pat_4655 (0.038140845814...
3,ther_21,Coronary artery bypass graft,Surgical procedures,cured with success on pat_5695 (0.017236971078...
4,ther_45,Liver transplant,Surgical procedures,cured with success on pat_7468 (0.013686589419...


In [50]:
i.to_csv("../../results/results.txt", index=None, sep='-', mode='a')

In [87]:
count = 1
with open("../../results/results.txt", 'a') as f:
        f.write("Recommended therapies for {patient_id} to treat condition {cond_id}: \n")
        for idx,row in i.iterrows():
            #with open("../../results/results.txt", 'a') as f:
            f.write(f'{count}°: {row.id}  |  {row.name}  |  {row.type}  |  {row.details} \n')
            count += 1
        f.write("-----------------------------------------------------------------------------------------------------------------------------------------\n")


In [70]:
with open('file1.txt', mode='w') as file_object:
        print(i, file=file_object)

In [91]:
reccomended_ther = pd.DataFrame(columns = ['id', 'name', 'type'])

In [92]:
reccomended_ther

,id,name,type


In [96]:
ther = therapies_df[therapies_df.id == "ther_0"]
ther


,id,name,type
0,ther_0,Abortion,Surgical procedures


In [108]:
import warnings
warnings.filterwarnings('ignore')

reccomended_ther =reccomended_ther.append(ther)
reccomended_ther

,id,name,type
0,ther_0,Abortion,Surgical procedures
0,ther_0,Abortion,Surgical procedures
0,ther_0,Abortion,Surgical procedures
0,ther_0,Abortion,Surgical procedures
0,ther_0,Abortion,Surgical procedures


In [105]:
len(reccomended_ther)

3

In [106]:
reccomended_ther.shape

(3, 3)